In [1]:
import pandas as pd

#### Split-Apply-Combine

1. Data is split into separate parts based on key(s).
2. A function is applied to each part of the data.
3. The results from each part are combined to create a new data set.

In [2]:
df = pd.read_csv('../data/gapminder.tsv', sep='\t')

In [3]:
df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [4]:
# Calculate the average life expectancy for each year.
df.groupby('year')['lifeExp'].mean()

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

In [5]:
df.groupby(['year', 'continent'])['lifeExp'].mean()

year  continent
1952  Africa       39.135500
      Americas     53.279840
      Asia         46.314394
      Europe       64.408500
      Oceania      69.255000
1957  Africa       41.266346
      Americas     55.960280
      Asia         49.318544
      Europe       66.703067
      Oceania      70.295000
1962  Africa       43.319442
      Americas     58.398760
      Asia         51.563223
      Europe       68.539233
      Oceania      71.085000
1967  Africa       45.334538
      Americas     60.410920
      Asia         54.663640
      Europe       69.737600
      Oceania      71.310000
1972  Africa       47.450942
      Americas     62.394920
      Asia         57.319269
      Europe       70.775033
      Oceania      71.910000
1977  Africa       49.580423
      Americas     64.391560
      Asia         59.610556
      Europe       71.937767
      Oceania      72.855000
1982  Africa       51.592865
      Americas     66.228840
      Asia         62.617939
      Europe       72.80640

In [6]:
# group by continent and year and describe each group
continent_describe = df.groupby('continent').lifeExp.describe()
print(continent_describe)

           count       mean        std     min       25%      50%       75%  \
continent                                                                     
Africa     624.0  48.865330   9.150210  23.599  42.37250  47.7920  54.41150   
Americas   300.0  64.658737   9.345088  37.579  58.41000  67.0480  71.69950   
Asia       396.0  60.064903  11.864532  28.801  51.42625  61.7915  69.50525   
Europe     360.0  71.903686   5.433178  43.585  69.57000  72.2410  75.45050   
Oceania     24.0  74.326208   3.795611  69.120  71.20500  73.6650  77.55250   

              max  
continent          
Africa     76.442  
Americas   80.653  
Asia       82.603  
Europe     81.757  
Oceania    81.235  


### Transform
When we transform data, we pass values from our dataframe into a function. The function then "transforms" the data. `transform` takes multiple values and returns a one-to-one transformation of the values (i.e., it does not reduce the amount of data, like `aggregate`).

In [7]:
# Python function to calculate a z-score (x - mu / sigma)
# x is a data point in our data set
# mu is the average of our data set
# sigma is the standard deviation
def my_zscore(x):
    return((x - x.mean()) / x.std())

In [8]:
# transform data by group
transform_z = df.groupby('year').lifeExp.transform(my_zscore)

In [9]:
transform_z

0      -1.656854
1      -1.731249
2      -1.786543
3      -1.848157
4      -1.894173
          ...   
1699   -0.081621
1700   -0.336974
1701   -1.574962
1702   -2.093346
1703   -1.948180
Name: lifeExp, Length: 1704, dtype: float64

In [10]:
# Original data set and the transform data set have the same number of rows...
print(df.shape)
print(transform_z.shape)

(1704, 6)
(1704,)


In [11]:
# Another example using the scipy built-in zscore function
from scipy.stats import zscore

# Grouped score
sp_z_grouped = df.groupby('year').lifeExp.transform(zscore)

# Non-grouped score
sp_z_nogroup = zscore(df.lifeExp)

# The z scores are not the same because one is calculated by group, and one is not.
print(sp_z_grouped)
print(sp_z_nogroup)

0      -1.662719
1      -1.737377
2      -1.792867
3      -1.854699
4      -1.900878
          ...   
1699   -0.081910
1700   -0.338167
1701   -1.580537
1702   -2.100756
1703   -1.955077
Name: lifeExp, Length: 1704, dtype: float64
[-2.37533395 -2.25677417 -2.1278375  ... -0.98080452 -1.5089415
 -1.23805834]


#### Missing value example...

In [12]:
import seaborn as sns
import numpy as np

# set seed so that results are deterministic
np.random.seed(42)

# sample 10 rows from tips dataset
tips_10 = sns.load_dataset('tips').sample(10)

# randomly pick 4 'total_bill' values and make them missing values
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN

print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


We can use the `groupby` statement to calculate a statistic to fill in missing values, using the `transform` method.

In [13]:
count_sex = tips_10.groupby('sex').count()
count_sex

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,4,7,7,7,7,7
Female,2,3,3,3,3,3


In [14]:
# Calculate a group average to fill in the missing values.
def fill_na_mean(x):
    avg = x.mean()
    return(x.fillna(avg))

# calculate a mean 'total_bill' by 'sex'
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)

# assign to a new column in the original data (we could also have replaced the original column)
tips_10['fill_total_bill'] = total_bill_group_mean

print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


#### Filtering

In [15]:
tips = sns.load_dataset('tips')
tips.shape

(244, 7)

In [17]:
tips['size'].value_counts()

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64

In [18]:
# filter out the three infrequent observations
tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count() >=30)
tips_filtered.shape

(231, 7)

In [19]:
tips_filtered['size'].value_counts()

2    156
3     38
4     37
Name: size, dtype: int64

#### Grouped calculations involving multiple variables

In [21]:
# Save just the grouped object (type DataFrameGroupBy)
grouped = tips_10.groupby('sex')

# Calculate the mean on relevant (numeric) columns
grouped.mean()

,total_bill,tip,size,fill_total_bill
sex,,,,
Male,17.9525,2.875714,2.571429,17.9525
Female,13.9300,2.506667,2.000000,13.9300


In [22]:
# We can also perform grouped calculations on a selected group using 'get_group'
female = grouped.get_group('Female')
female

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
198,NaN,2.00,Female,Yes,Thur,Lunch,2,13.93
124,12.48,2.52,Female,No,Thur,Lunch,2,12.48
101,15.38,3.00,Female,Yes,Fri,Dinner,2,15.38


#### Multiple groups (and flattening the results)

In [23]:
bill_sex_time = tips_10.groupby(['sex', 'time'])
group_avg = bill_sex_time.mean()
group_avg

total_bill       tip      size  fill_total_bill
sex    time                                                   
Male   Lunch    28.440000  2.560000  2.000000        28.440000
       Dinner   14.456667  2.928333  2.666667        16.204583
Female Lunch    12.480000  2.260000  2.000000        13.205000
       Dinner   15.380000  3.000000  2.000000        15.380000

In [24]:
# To get a regular flat dataframe, call 'reset_index' on the results.
group_flat = tips_10.groupby(['sex', 'time']).mean().reset_index()
group_flat

,sex,time,total_bill,tip,size,fill_total_bill
0,Male,Lunch,28.440000,2.560000,2.000000,28.440000
1,Male,Dinner,14.456667,2.928333,2.666667,16.204583
2,Female,Lunch,12.480000,2.260000,2.000000,13.205000
3,Female,Dinner,15.380000,3.000000,2.000000,15.380000


### Working with a MultiIndex

In [26]:
# Epidemiological simulation data on influenza cases in Chicago
intv_df = pd.read_csv('../data/epi_sim.txt')
intv_df.shape

(9434653, 6)

In [27]:
intv_df.sample(5)

,ig_type,intervened,pid,rep,sid,tr
7564968,3,73,292176661,1,201,0.000166
7511501,2,68,289454022,1,201,0.000166
433232,2,76,288508915,1,201,0.000135
7687546,2,59,293115682,1,201,0.000166
477893,2,77,291469022,1,201,0.000135


In [28]:
count_only = intv_df.groupby(['rep', 'intervened', 'tr'])['ig_type'].count()

In [29]:
count_only.head(10)

rep  intervened  tr      
0    8           0.000166    1
     9           0.000152    3
                 0.000166    1
     10          0.000152    1
                 0.000166    1
     12          0.000152    3
                 0.000166    5
     13          0.000152    1
                 0.000166    3
     14          0.000152    3
Name: ig_type, dtype: int64